In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, RepeatedKFold
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [9]:
teste = pd.read_csv("/content/sample_data/california_housing_test.csv")
treino = pd.read_csv("/content/sample_data/california_housing_train.csv")

In [10]:
teste.head(2)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0


In [11]:
treino.head(2)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0


In [12]:
x = treino.drop("median_house_value", axis=1)
y = treino['median_house_value']

In [13]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state=0)

In [14]:
x_treino.shape

(11900, 8)

In [15]:
arvore = XGBRegressor(learning_rate=0.08, n_estimators=1300, max_depth=6, min_child_weight=1, subsample=0.75,
                      colsample_bynode=0.5, random_state=0, booster="gbtree", objective='reg:squarederror')

In [16]:
arvore.fit(x_treino, y_treino)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=0.5, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=1300,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.75, verbosity=1)

In [17]:
previsao = arvore.predict(x_teste)
print(np.sqrt(mean_squared_error(y_teste, previsao)))

45720.79359014327


In [18]:
arvore.score

<bound method RegressorMixin.score of XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=0.5, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=1300,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.75, verbosity=1)>

In [20]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(arvore, x_treino, y_treino, scoring='neg_mean_absolute_error', cv=cv)

In [24]:
scores = np.absolute(scores)
print('Media: %.3f, desvio padrão: (%.3f)' % (scores.mean(), scores.std()))

Media: 31047.635, desvio padrão: (880.949)
